<a href="https://colab.research.google.com/github/ammn23/recsystem/blob/main/recipe_recsystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
#

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
recipes=pd.read_csv('/content/drive/MyDrive/recipes_w_search_terms.csv')

In [41]:
recipes.head()

,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms
0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to ser...","['water', 'grits', 'salt', 'cheddar cheese', '...","[""4 cups water"",""1 cup uncooked old f...",1 (155 g),8,"['I a sauce pan, bring water to a boil; slowly...","['time-to-make', 'course', 'main-ingredient', ...","{'diabetic', 'low-calorie', 'vegetarian', 'low..."
1,232037,Simple Shrimp and Andouille Jambalaya,"Simple, easy and very tasty for when you are i...","['onion', 'red bell pepper', 'garlic cloves', ...","[""1 medium onion, chopped coarse "",""1 m...",1 (366 g),4,"['In a food processor, pulse the onion, red pe...","['60-minutes-or-less', 'time-to-make', 'course...","{'dinner', 'shrimp'}"
2,41090,black-and-white bean salad,NaN,"['white beans', 'canned black beans', 'tomatoe...","[""1 cup canned white beans, rinsed and dra...",1 (807 g),1,"['In a large bowl, combine beans, tomato, onio...","['15-minutes-or-less', 'time-to-make', 'course...","{'vegetarian', 'salad', 'side', 'dinner', 'veg..."
3,60656,Crock Pot Italian Zucchini,This is a good recipe for weight watchers. It ...,"['zucchini', 'yellow squash', 'diced tomatoes'...","[""2 zucchini, sliced "",""2 small yel...",1 (244 g),4,['Put all ingredients in the crock pot and coo...,"['weeknight', 'time-to-make', 'course', 'main-...","{'side', 'vegetarian', 'italian'}"
4,232047,Beef Stew With Dried Cherries,This is a fabulous stew that came from one of ...,"['beef stew meat', 'flour', 'salt', 'allspice'...","[""3 lbs beef stew meat"",""3 tablespoons ...",1 (358 g),8,"['Preheat oven to 350°F.', ""Cut beef into 1 in...","['time-to-make', 'course', 'main-ingredient', ...",{'dinner'}


In [42]:
recipes['diabetic'] = recipes['tags'].apply(lambda x: 1 if 'diabetic' in x else 0)

In [43]:
recipes.head()

,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms,diabetic
0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to ser...","['water', 'grits', 'salt', 'cheddar cheese', '...","[""4 cups water"",""1 cup uncooked old f...",1 (155 g),8,"['I a sauce pan, bring water to a boil; slowly...","['time-to-make', 'course', 'main-ingredient', ...","{'diabetic', 'low-calorie', 'vegetarian', 'low...",1
1,232037,Simple Shrimp and Andouille Jambalaya,"Simple, easy and very tasty for when you are i...","['onion', 'red bell pepper', 'garlic cloves', ...","[""1 medium onion, chopped coarse "",""1 m...",1 (366 g),4,"['In a food processor, pulse the onion, red pe...","['60-minutes-or-less', 'time-to-make', 'course...","{'dinner', 'shrimp'}",0
2,41090,black-and-white bean salad,NaN,"['white beans', 'canned black beans', 'tomatoe...","[""1 cup canned white beans, rinsed and dra...",1 (807 g),1,"['In a large bowl, combine beans, tomato, onio...","['15-minutes-or-less', 'time-to-make', 'course...","{'vegetarian', 'salad', 'side', 'dinner', 'veg...",0
3,60656,Crock Pot Italian Zucchini,This is a good recipe for weight watchers. It ...,"['zucchini', 'yellow squash', 'diced tomatoes'...","[""2 zucchini, sliced "",""2 small yel...",1 (244 g),4,['Put all ingredients in the crock pot and coo...,"['weeknight', 'time-to-make', 'course', 'main-...","{'side', 'vegetarian', 'italian'}",0
4,232047,Beef Stew With Dried Cherries,This is a fabulous stew that came from one of ...,"['beef stew meat', 'flour', 'salt', 'allspice'...","[""3 lbs beef stew meat"",""3 tablespoons ...",1 (358 g),8,"['Preheat oven to 350°F.', ""Cut beef into 1 in...","['time-to-make', 'course', 'main-ingredient', ...",{'dinner'},0


In [44]:
recipes['id']=recipes['id'].astype(str)

In [45]:
recipes.dropna(subset=['name'], inplace=True)

In [46]:
recipes.dropna(subset=['description'], inplace=True)

In [47]:
print('duplicate rows',recipes.duplicated().sum())

duplicate rows 0


In [48]:
recipes['dairy-free'] = recipes['tags'].apply(lambda x: 1 if 'dairy-free' in x else 0)
recipes['gluten-free'] = recipes['tags'].apply(lambda x: 1 if 'gluten-free' in x else 0)
recipes['low-carb'] = recipes['tags'].apply(lambda x: 1 if 'low-carb' in x else 0)
recipes['vegan'] = recipes['tags'].apply(lambda x: 1 if 'vegan' in x else 0)
recipes['vegetarian'] = recipes['tags'].apply(lambda x: 1 if 'vegetarian' in x else 0)
recipes['low-calorie'] = recipes['tags'].apply(lambda x: 1 if 'low-calorie' in x else 0)
recipes['easy'] = recipes['tags'].apply(lambda x: 1 if 'easy' in x else 0)
recipes['easy'] = recipes['tags'].apply(lambda x: 1 if 'beginner-cook' in x else 0)
recipes['easy'] = recipes['tags'].apply(lambda x: 1 if 'beginner-cook' in x else 0)



In [49]:

recipes['time'] = recipes['tags'].apply(lambda x: 15 if '15-minutes-or-less' in x else (30 if '30-minutes-or-less' in x else (60 if '60-minutes-or-less' in x else 100)))

In [50]:
def categorize_meal(search_terms, name):
    if 'breakfast' in search_terms or 'brunch' in search_terms:
        return 'breakfast'
    elif 'lunch' in search_terms or 'soup' in search_terms or 'chicken' in search_terms or 'stew' in search_terms or 'beef' in search_terms:
        return 'lunch'
    elif 'dinner' in search_terms or 'soup' in search_terms or 'chicken' in search_terms or 'stew' in search_terms or 'beef' in search_terms:
        return 'dinner'
    elif 'snack' in search_terms or 'appetizer' in search_terms:
        return 'snack'
    elif 'drink' in search_terms or 'drink' in name or 'Coffee' in name or 'Cocktail' in name:
        return 'drink'
    elif 'side' in search_terms or 'salad' in search_terms:
        return 'side'
    elif 'dessert' in search_terms:
        return 'dessert'
    elif 'bread' in search_terms:
        return 'bread'
    elif 'sauce' in search_terms or 'Sauce' in name:
        return 'sauce'
    else:
        return 'other'

recipes['meal'] = recipes.apply(lambda row: categorize_meal(row['search_terms'], row['name']), axis=1)

In [51]:
recipes['meal'].value_counts()

meal
dinner       140348
lunch        110243
dessert       82024
side          42875
other         32150
breakfast     30810
snack         25791
bread         13724
sauce          4855
drink          2542
Name: count, dtype: int64

In [52]:
meals_to_keep = ['dinner', 'lunch', 'dessert', 'side', 'snack', 'breakfast']
recipes = recipes[recipes['meal'].isin(meals_to_keep)]

In [53]:
recipes['meal'].value_counts()

meal
dinner       140348
lunch        110243
dessert       82024
side          42875
breakfast     30810
snack         25791
Name: count, dtype: int64

In [54]:
recipes.shape

(432091, 20)

In [55]:
recipes=recipes[recipes['easy']==0]
recipes.drop('easy',axis=1,inplace=True)
recipes.head()

,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms,diabetic,dairy-free,gluten-free,low-carb,vegan,vegetarian,low-calorie,time,meal
0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to ser...","['water', 'grits', 'salt', 'cheddar cheese', '...","[""4 cups water"",""1 cup uncooked old f...",1 (155 g),8,"['I a sauce pan, bring water to a boil; slowly...","['time-to-make', 'course', 'main-ingredient', ...","{'diabetic', 'low-calorie', 'vegetarian', 'low...",1,0,0,1,0,1,1,100,side
1,232037,Simple Shrimp and Andouille Jambalaya,"Simple, easy and very tasty for when you are i...","['onion', 'red bell pepper', 'garlic cloves', ...","[""1 medium onion, chopped coarse "",""1 m...",1 (366 g),4,"['In a food processor, pulse the onion, red pe...","['60-minutes-or-less', 'time-to-make', 'course...","{'dinner', 'shrimp'}",0,0,0,0,0,0,0,60,dinner
4,232047,Beef Stew With Dried Cherries,This is a fabulous stew that came from one of ...,"['beef stew meat', 'flour', 'salt', 'allspice'...","[""3 lbs beef stew meat"",""3 tablespoons ...",1 (358 g),8,"['Preheat oven to 350°F.', ""Cut beef into 1 in...","['time-to-make', 'course', 'main-ingredient', ...",{'dinner'},0,0,0,0,0,0,0,100,dinner
5,232050,Hot Sweet Almond Brittle,This is one of our standard holiday gift recip...,"['slivered almonds', 'cider vinegar', 'sugar',...","[""12 ounces slivered almonds"",""1/4 cup ...",1 (832 g),1,['Preheat oven to 375°F Place almonds in sing...,"['time-to-make', 'course', 'preparation', 'des...",{'dessert'},0,0,0,0,0,0,0,100,dessert
6,232076,Retro Chicken &amp; Chips Casserole,"From Cooking Light Magazine, 11/05. This is co...","['chicken breasts', 'green onion', 'red bell p...","[""4 cups roasted chopped chicken breasts"",...",1 (85 g),6,"['In large bowl, combine chicken, green onion,...","['30-minutes-or-less', 'time-to-make', 'course...","{'casserole', 'dinner', 'chicken'}",0,0,0,0,0,0,0,30,lunch


In [56]:
recipes.head()

,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms,diabetic,dairy-free,gluten-free,low-carb,vegan,vegetarian,low-calorie,time,meal
0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to ser...","['water', 'grits', 'salt', 'cheddar cheese', '...","[""4 cups water"",""1 cup uncooked old f...",1 (155 g),8,"['I a sauce pan, bring water to a boil; slowly...","['time-to-make', 'course', 'main-ingredient', ...","{'diabetic', 'low-calorie', 'vegetarian', 'low...",1,0,0,1,0,1,1,100,side
1,232037,Simple Shrimp and Andouille Jambalaya,"Simple, easy and very tasty for when you are i...","['onion', 'red bell pepper', 'garlic cloves', ...","[""1 medium onion, chopped coarse "",""1 m...",1 (366 g),4,"['In a food processor, pulse the onion, red pe...","['60-minutes-or-less', 'time-to-make', 'course...","{'dinner', 'shrimp'}",0,0,0,0,0,0,0,60,dinner
4,232047,Beef Stew With Dried Cherries,This is a fabulous stew that came from one of ...,"['beef stew meat', 'flour', 'salt', 'allspice'...","[""3 lbs beef stew meat"",""3 tablespoons ...",1 (358 g),8,"['Preheat oven to 350°F.', ""Cut beef into 1 in...","['time-to-make', 'course', 'main-ingredient', ...",{'dinner'},0,0,0,0,0,0,0,100,dinner
5,232050,Hot Sweet Almond Brittle,This is one of our standard holiday gift recip...,"['slivered almonds', 'cider vinegar', 'sugar',...","[""12 ounces slivered almonds"",""1/4 cup ...",1 (832 g),1,['Preheat oven to 375°F Place almonds in sing...,"['time-to-make', 'course', 'preparation', 'des...",{'dessert'},0,0,0,0,0,0,0,100,dessert
6,232076,Retro Chicken &amp; Chips Casserole,"From Cooking Light Magazine, 11/05. This is co...","['chicken breasts', 'green onion', 'red bell p...","[""4 cups roasted chopped chicken breasts"",...",1 (85 g),6,"['In large bowl, combine chicken, green onion,...","['30-minutes-or-less', 'time-to-make', 'course...","{'casserole', 'dinner', 'chicken'}",0,0,0,0,0,0,0,30,lunch


In [57]:
recipes=recipes[recipes['time']!=100]

In [58]:
recipes.shape

(260824, 19)

In [59]:
recipes['meal'].value_counts()

meal
dinner       78544
lunch        62441
dessert      50280
side         29439
breakfast    21456
snack        18664
Name: count, dtype: int64

In [60]:
# Filter the dataframe
filtered_recipes = recipes[(recipes['meal'] == 'dinner') | (recipes['meal'] == 'lunch')]

# Sample 3000 recipes
sampled_recipes = filtered_recipes.sample(n=3000, random_state=42) # Set random_state for reproducibility

print(sampled_recipes)

            id                                   name  \
310855  249086                 Ground Beef Enchiladas   
265229  347317                     Turkey Noodle Bake   
142364  514214  Sous-Vide Double Cut Pork Chops for 2   
490612  196917                         Mom's Gazpacho   
365600  434451                       Tofu Bourguignon   
...        ...                                    ...   
424926  470837                  *no Cook Summer Pasta   
246772    9935               Easy Hamburger Casserole   
14914   275026                 Impossible Seafood Pie   
355582  137906                        Chicken Risotto   
75772   200127              Thai Coconut Chicken Soup   

                                              description  \
310855                             Ground Beef Enchiladas   
265229  This is from an old Better Homes & Garden Cook...   
142364  Entered for safe-keeping, from J. Kenji López-...   
490612  My Mom made this when I was a kid and hated to...   
365600   V

In [61]:
sampled_recipes['low-calorie'].value_counts()

low-calorie
0    2547
1     453
Name: count, dtype: int64

In [62]:
filtered_recipes = recipes[(recipes['meal'] == 'dessert') | (recipes['meal'] == 'side') | (recipes['meal'] == 'breakfast') | (recipes['meal'] == 'snack')]

sampled_recipes2 = filtered_recipes.sample(n=1000, random_state=42)

print(sampled_recipes2)

            id                                               name  \
830     521273                        Mediterranean Rotinin Salad   
463874  454973   Cranberry Ham &amp; Cheese Appetizers (Or Wraps)   
197775  299836                               Pina Colada Smoothie   
84978   182663                        Chocolate Cheesecake Chunks   
207366  365989                               Chocolate Sheet Cake   
...        ...                                                ...   
292096  305536                        Tyropites (Cheese Pastries)   
287585  437696                            Mini Cheeseburger Salad   
227858  390892                Incredibly Easy Lemon Cream Cookies   
485209  408522                 Chocolate Panini (Weight Watchers)   
390958   26306  Copycat  Green Giant Niblets Corn in Butter Sauce   

                                              description  \
830     This is somewhere else on Food.com... but good...   
463874  Although these are colorful for the yulet

In [63]:
combined_recipes = pd.concat([sampled_recipes, sampled_recipes2])

print(combined_recipes)

            id                                               name  \
310855  249086                             Ground Beef Enchiladas   
265229  347317                                 Turkey Noodle Bake   
142364  514214              Sous-Vide Double Cut Pork Chops for 2   
490612  196917                                     Mom's Gazpacho   
365600  434451                                   Tofu Bourguignon   
...        ...                                                ...   
292096  305536                        Tyropites (Cheese Pastries)   
287585  437696                            Mini Cheeseburger Salad   
227858  390892                Incredibly Easy Lemon Cream Cookies   
485209  408522                 Chocolate Panini (Weight Watchers)   
390958   26306  Copycat  Green Giant Niblets Corn in Butter Sauce   

                                              description  \
310855                             Ground Beef Enchiladas   
265229  This is from an old Better Homes & Garden

In [64]:
recipes=combined_recipes

In [65]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [66]:
def normalize(ingredient):
    ingredient = ingredient.lower()
    ingredient = re.sub(r'[^a-zA-Z0-9\s]', '', ingredient)
    words = ingredient.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    norm_ingr= ' '.join(words)
    return norm_ingr

normalized = [normalize(ingredient) for ingredient in recipes['ingredients']]


In [69]:
recipes.drop(['search_terms','ingredients','tags'],axis=1,inplace=True)

In [76]:
recipes[(recipes['meal']=='lunch' )| (recipes['meal']=='dinner')]['diabetic'].value_counts()

diabetic
0    2925
1      75
Name: count, dtype: int64

In [77]:
vectorizer = TfidfVectorizer()
ingredient_matrix = vectorizer.fit_transform(normalized)